In [ ]:
# how much energy does something take:
/usr/bin/time -v 

## Manually comparing what each program found
- skipmer vs kmer: Marginally better, may not be worth extra compute
- Different mappers for RGI: no difference


So for now, compare kmer to RGI. There are 11 things sourmash found, but not RGI and 15 things other way around. 72 are in common. Make db of these 98 sequences, and map reads or do a sourmash gather. 


After this, will need to extract the reads that map to refs smash or RGI cant find, and blast these.

In [ ]:
# use filterby name by bbmap
# allows for substring matching
mamba activate bbmap
filterbyname.sh \
in=../../resources/database/CARD/nucleotide_fasta_protein_homolog_model.fasta \
out=found_seqs.fa names=found_seqs.txt include=t substring=t

In [ ]:
# do smash part
mamba activate branchwater-skipmer
sourmash sketch dna \
found_seqs.fa \
--singleton -p k=21,scaled=100 \
-o found_seqs.smash.zip

In [ ]:
# sketch the rgi db and fmg against the raw reads that are sketched
sourmash scripts fastgather \
../smash/signatures/SRR5240728.metag.zip \
found_seqs.smash.zip \
-k 21 --scaled 100 -t 0 \
-c 30 -o SRR5240728.x.foundseqs.csv

In [ ]:
# the rgi conda env has kma in it (SRR5240728)
# index db
kma index -i found_seqs.fa -o kmadb

# run kma
kma -mem_mode -ex_mode -1t1 -vcf \
-ipe /group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/atlas/atlas_SRR5240728/SRR5240728/sequence_quality_control/SRR5240728_QC_R*.fastq.gz \
-t 30 -t_db kmadb \
-o SRR5240728.x.kmadb -sam



In [ ]:
# Next step:
# Extracting reads related to kmers
# first need to extract fast we want to compare to
filterbyname.sh in=../../resources/database/CARD/nucleotide_fasta_protein_homolog_model.fasta \
out=extract_reads/found1.fa names=foundbyone.txt include=t substring=t


srun --account=ctbrowngrp -p bmm -J extractread -t 168:00:00 -c 1 --mem=50gb --pty bash

# now we can match the kmers from the fq file to these sequences, and extract reads that match.
mamba activate branchwater-skipmer
sourmash sig kmers --signatures ../../smash/signatures/SRR5240728.metag.zip \
-k 21 --sequences found1.fa \
--save-sequences recovered_reads_7days.fasta --save-kmers kmer-matches_7day.csv

In [ ]:
# Can use the kma consensus sequences too for blast
